# Striatum Scaffold Model

The purpose of this code is to setup a scaffold model of the striatum and run it.

1. Download the code from the collab storage
2. Place neurons in a 3D space based on a config file, in this case: 
   Network-striatum-cube-v7-channels-100-15.json  (A small 100 neuron network)

3. Perform touch detection to place the synapses. For neurons like ChIN that currently do not
   have axons in their reconstructed morphologies we use probability clouds.
   This step also includes pruning of synapses.

4. Plot statistics for the network connectivity

5. Setup and run the simulation using the Neuron simulator

6. Plot voltage traces, proof of concept that the network simulation runs.


### First we need to download the source code from the collab and unpack it

In [ ]:
import os
storage = get_bbp_client().document
storageBasePath = get_collab_storage_path()
storage.download_file(storageBasePath + os.path.sep + "Striatum-scaffold-network-use-case-2018.tar.bz2", \
                      "Striatum-scaffold-network-use-case-2018.tar.bz2")

import tarfile
tar = tarfile.open("Striatum-scaffold-network-use-case-2018.tar.bz2")
tar.extractall()
tar.close()

os.chdir('StriatumScaffold2018')

# This is some magic to make the plots show up in the notebook
%matplotlib inline

In [ ]:
ls

### Place neurons in 3D space

A configuration file specifies the network, such as the number of neurons of each type, and information about which other neuron types they can connect to. Based on this information the 3D positions and rotations of each neuron is first specified. The d-min algorithm is used, which randomly places the neurons in 3D space, but allows for an exclusion zone around each neuron. Special care has also been taken to avoid edge effects.

The resulting positions are stored in a pickle file.

In [ ]:
# Place the neurons in 3D space (includes rotation)
from Network_place_neurons import NetworkPlaceNeurons

# The config file specifies what neurons to place, and which neuron types they connect to
# See makeStratiumConfigFile.py (included in the tar file) for more information and to generate your own
# network config file if you want to change something.
configFile="config/Network-striatum-cube-v7-channels-100-15.json"

# Name of the output file with neuron positions
positionFile="save/network-positions-file.pickle"

# This places the neurons in 3D space using the D-min algorithm (ie an exclusion zone around each neuron)
npn = NetworkPlaceNeurons(config_file=configFile,logFile=None)
npn.writeData(positionFile)

### Creating network connectivity

Once the neurons are placed in 3D space we want to find the connections between them. We look for places where the axon and dendrites of different neurons are in close apposition and place a putative synapse if the neuron types can have connections between them. These putative synapses are then pruned down based on a set of rules specified in the original network config file.

In [ ]:
# Create connectivity (step 1, initialisation)
from Network_connect import NetworkConnect

# This is the file the network connectivity is written to
saveFile = 'save/Network-striatum-100.hdf5'

ncMaster = NetworkConnect(position_file=positionFile, \
                          config_file=configFile,
                          logFile=None,
                          save_file=saveFile)

The cell below runs the actual touch detection. This can take a bit of time (~ 10 minutes).

In [ ]:
# This step takes a while to run. It places synapses where axons and dendrites are in close proximity.
dView = None # Not running in parallel in this example
ncMaster.findGapJunctions(dView)
ncMaster.findSynapses(dView)
ncMaster.saveHDF5()

### Plot statistics of network circuitry

Here we plot a network connection statistics, such as number of synapses between different types of connected neurons. We also plot connection probability as a function of distance.

In [ ]:
# Extract some statistics from the network connectivity
from Network_plot_statistics import Network_plot_statistics
network_plot = Network_plot_statistics(network_file=saveFile)
network_plot.plotNumConnections()
# network_plot.distanceDistribution()
network_plot.connectionProbability(preType="MSD1",postType="MSD2")

### Compile the channel modules needed for the network simulation

In [ ]:
# compile the channel mod files
import commands
commands.getstatusoutput('/usr/local/nrn/x86_64/bin/nrnivmodl')

In [ ]:
ls

In [ ]:
! mkdir -p save/traces

### Run the network simulation using neuron

This step can take a bit of time (~10 minutes). The result is stored in the save/traces subdirectory.

In [ ]:
# Simulate the network using Neuron
from mpi4py import MPI
from neuron import h, gui
from Network_simulate import NetworkSimulate
pc = h.ParallelContext()
sim = NetworkSimulate(saveFile)
sim.addExternalInput(freq=100)
sim.addRecording()
sim.run(200.0)
spikeFile='save/traces/network-output-spikes.txt'
voltFile='save/traces/network-voltage.csv'
sim.writeSpikes(spikeFile)
sim.writeVoltage(voltFile)

### Plot voltage traces

In [ ]:
# Plot the voltage traces from the simulation
from Network_plot_traces import NetworkPlotTraces
npt = NetworkPlotTraces(voltFile)
npt.plotTraces()

In [ ]:
ls save